## Bascis

Libs

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import os

Functions

In [2]:
def le_nome_cidade(csv):
    with open(csv, encoding='utf-8') as f:
        primeira_linha = f.readline()
        nome_cidade = primeira_linha.split(':')[1].strip()  # Ajuste conforme o formato real da linha
    return nome_cidade

def get_file_paths(folder_path=None):
    folder_path = 'C:\\Users\\tiago\\UFSC (local)\\Prologis\\Panic buyng\\Dados ambientais (INMET)'

    file_paths = []
    nomes = []
    # Percorre todos os arquivos e subdiretórios
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            # Cria o caminho completo do arquivo
            full_path = os.path.join(root, file)
            file_paths.append(full_path)
            le_nome_cidade(full_path)
            nomes.append(le_nome_cidade(full_path))

    return file_paths, nomes

def data_analisys(nome, csv):
    
    #le e filtra arquivo
    db_null = pd.read_csv(csv, sep=';', skiprows=10)
    db_null = db_null[['Data Medicao', 'Hora Medicao','PRECIPITACAO TOTAL, HORARIO(mm)','VENTO, RAJADA MAXIMA(m/s)','RADIACAO GLOBAL(Kj/m²)']]

    #DB
    db = db_null.fillna(0)

    # Trata os dados
    db['Data Medicao'] = pd.to_datetime(db['Data Medicao'], errors='coerce')
    colunas_float = ['PRECIPITACAO TOTAL, HORARIO(mm)', 'VENTO, RAJADA MAXIMA(m/s)', 'RADIACAO GLOBAL(Kj/m²)']
    for col in colunas_float:
        db[col] = db[col].astype(str).str.replace(',', '.').astype(float)
    db['AnoMes'] = db['Data Medicao'].dt.to_period('M')

    # Análise estatística básica
    dados_estatisticos = db.describe(include='all')

    info = {
    'nome': nome,
    'qtd_nulos_chuva': db_null.isnull().sum()['PRECIPITACAO TOTAL, HORARIO(mm)'],
    'qtd_nulos_vento': db_null.isnull().sum()['VENTO, RAJADA MAXIMA(m/s)'],
    'chuva_total': float(db['PRECIPITACAO TOTAL, HORARIO(mm)'].sum())
    }
    
    info_df = pd.DataFrame([info])

    # Chuva diaria 24 horas)
    db_diario = db.groupby('Data Medicao')['PRECIPITACAO TOTAL, HORARIO(mm)'].sum().reset_index()

    # Soma a precipitação por mês
    db_mensal = db.groupby('AnoMes')['PRECIPITACAO TOTAL, HORARIO(mm)'].sum().reset_index()

    return db, dados_estatisticos, info_df, db_diario, db_mensal

def plot_prec_diario_interativo(dataframe, nome):

    fig = go.Figure()

    # Garantindo que o eixo X seja tratado corretamente
    x = dataframe['Data Medicao'].astype(str)
    y = dataframe['PRECIPITACAO TOTAL, HORARIO(mm)']
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines+markers', name=nome))

    # Configurações de Layout
    fig.update_layout(
        title= f'Precipitação Total Diária - {nome}',
        xaxis_title='Periodo',
        yaxis_title='Precipitação (mm)',
        hovermode='x unified', # Mostra todos os dados ao passar o mouse em uma data
        legend_title='Localidades/Séries',
        template='plotly_white',
        xaxis=dict(tickangle=45)
    )

    fig.show(renderer="browser")

def plot_prec_mensal_interativo_multi(dataframes, labels):

    fig = go.Figure()

    for df, label in zip(dataframes, labels):
        # Garantindo que o eixo X seja tratado corretamente
        x = df['AnoMes'].astype(str)
        y = df['PRECIPITACAO TOTAL, HORARIO(mm)']
        
        # Adiciona cada linha individualmente
        fig.add_trace(go.Scatter(
            x=x, 
            y=y, 
            mode='lines+markers', 
            name=label,
            hovertemplate='<b>Data:</b> %{x}<br><b>Precipitação:</b> %{y}mm<extra></extra>'
        ))

    # Configurações de Layout
    fig.update_layout(
        title='Comparativo de Precipitação Total Mensal',
        xaxis_title='Ano-Mês',
        yaxis_title='Precipitação (mm)',
        hovermode='x unified', # Mostra todos os dados ao passar o mouse em uma data
        legend_title='Localidades/Séries',
        template='plotly_white',
        xaxis=dict(tickangle=45)
    )

    fig.show(renderer="browser")

Get files and names

In [3]:
lista_csv, nomes = get_file_paths()

In [4]:
lista_csv

['C:\\Users\\tiago\\UFSC (local)\\Prologis\\Panic buyng\\Dados ambientais (INMET)\\ararangua.csv',
 'C:\\Users\\tiago\\UFSC (local)\\Prologis\\Panic buyng\\Dados ambientais (INMET)\\chapeco.csv',
 'C:\\Users\\tiago\\UFSC (local)\\Prologis\\Panic buyng\\Dados ambientais (INMET)\\curitibanos.csv',
 'C:\\Users\\tiago\\UFSC (local)\\Prologis\\Panic buyng\\Dados ambientais (INMET)\\FLN 2020-01-01_2026-01-01.csv',
 'C:\\Users\\tiago\\UFSC (local)\\Prologis\\Panic buyng\\Dados ambientais (INMET)\\itajai.csv',
 'C:\\Users\\tiago\\UFSC (local)\\Prologis\\Panic buyng\\Dados ambientais (INMET)\\urussanga.csv']

In [5]:
nomes

['ARARANGUA', 'CHAPECO', 'CURITIBANOS', 'FLORIANOPOLIS', 'ITAJAI', 'URUSSANGA']

## Data Analytics

1) Le o arquivo
2) Pega só as variavies uteis: Data, Hora, PRECIPITACAO, VENTO, RADIACAO
3) Armazena a quantidade de valores nulos em cada coluna
4) Substitui nulos por 0
5) Data fixing: Data -> Datetime, str -> int
6) Coleta informações: estatisticas sobre o periodo, total precipitado 
7) Plota os graficos:  Precipitação Total Horária, Precipitação Total Diária, Precipitação mensal

### Aply

In [6]:

# 2. Agrupamento: Criar um dicionário onde a chave é a cidade e o valor é a lista de seus arquivos
files_by_city = {}
for path, nome in zip(lista_csv, nomes):
    if nome not in files_by_city:
        files_by_city[nome] = []
    files_by_city[nome].append(path)

dbs = {}

# Agrupamos os arquivos por cidade para processar um de cada vez
for nome, city_files in files_by_city.items():
    print(f"Processando dados de: {nome}")
    
    # Se você tem vários arquivos por cidade, aqui estamos pegando o primeiro [0]
    # Se quiser todos, precisaríamos de um pequeno ajuste para concatenar os arquivos
    path_do_arquivo = city_files[0] 
    
    # Chamada da sua função exatamente como você a definiu
    db, est, info, dbd, dbm = data_analisys(nome, path_do_arquivo)
    
    # Guardando na estrutura de Dicionário de Dicionários
    dbs[nome] = {
        "Dataframe": db,
        "Estatisticas": est,
        "Info": info,
        "DB_Days": dbd,
        "DB_Months": dbm
    }

print("Processamento concluído com sucesso!")

Processando dados de: ARARANGUA
Processando dados de: CHAPECO
Processando dados de: CURITIBANOS
Processando dados de: FLORIANOPOLIS
Processando dados de: ITAJAI
Processando dados de: URUSSANGA
Processamento concluído com sucesso!


Gráficos

In [7]:
dataframes = []
info_total = pd.DataFrame()
for nome in nomes:
    dataframes.append(dbs[nome]['DB_Months'])
    info_total = pd.concat([info_total, dbs[nome]['Info']])

plot_prec_mensal_interativo_multi(dataframes, nomes)

In [8]:
fln = dbs['FLORIANOPOLIS']['DB_Days']
plot_prec_diario_interativo(fln, 'Florianópolis')

In [9]:
#adicionar qtd picos de chuva
#adicionar mes-ano mais chuvoso
#adcionar qtd de dados totais
#add relação nulos/dado totais

# 20, 40 km/hr - considerar como picos de vento


info_total

,nome,qtd_nulos_chuva,qtd_nulos_vento,chuva_total
0,ARARANGUA,502,2862,9459.2
0,CHAPECO,1076,1103,11704.0
0,CURITIBANOS,10089,7173,7826.6
0,FLORIANOPOLIS,7312,7429,9730.8
0,ITAJAI,8499,8506,8051.6
0,URUSSANGA,7922,8423,9178.2


In [10]:
dbs['FLORIANOPOLIS']['Dataframe']

,Data Medicao,Hora Medicao,"PRECIPITACAO TOTAL, HORARIO(mm)","VENTO, RAJADA MAXIMA(m/s)",RADIACAO GLOBAL(Kj/m²),AnoMes
0,2020-01-01,0,0.0,6.1,-2.6,2020-01
1,2020-01-01,100,0.0,6.1,-2.5,2020-01
2,2020-01-01,200,0.0,7.4,-2.0,2020-01
3,2020-01-01,300,0.0,6.7,-2.0,2020-01
4,2020-01-01,400,0.0,7.9,-2.6,2020-01
...,...,...,...,...,...,...
52627,2026-01-01,1900,0.0,0.0,0.0,2026-01
52628,2026-01-01,2000,0.0,0.0,0.0,2026-01
52629,2026-01-01,2100,0.0,0.0,0.0,2026-01
52630,2026-01-01,2200,0.0,0.0,0.0,2026-01


## Resto

Tarefas

In [ ]:
# Jogar github o codigo

# classificar chuva, e vento em categorias (leve, moderada, forte)
# Buscar dados pandemia

# API automação para identificar a data e titulo de noticias com as plavras chaves (G1, CNN)

# Impacto do boca a boca e redes sociais no comportamento de compra durante eventos climáticos extremos